# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [15]:
browser = webdriver.Chrome(executable_path='/Users/tommy84729/python/1st-PyCrawlerMarathon/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

In [14]:
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    print('網頁更新中')
    time.sleep(1)

    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("到達頁面底端")
        break

    last_height = new_height

In [16]:
last_height = browser.execute_script("return document.body.scrollHeight")
last_height

6830

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [11]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 廣華-KY公布1月營收4.74億元　年減27%！積極發揮全球接單策略　優化訂單結構

▲廣華控股有限公司（廣華-KY）上市主體內主要營運據點為東莞廣澤汽車飾件有限公司、武漢廣佳汽車飾件有限公司，以及其他分公司。（圖／翻攝自東莞廣澤）
記者楊絡懸／台北報導
汽車飾板配件大廠「廣華-KY」（1338）公布2020年1月合併營收為新台幣4.74億元，較去年同期減少27.05%。廣華表示，過年長假導致工作天數較少，影響公司整體出貨表現，惟公司積極擴大於汽車內飾件市場的份額，持續優化訂單結構，並積極爭取全球性的車款訂單，1月營收年減率幅度能小於工作天數減少30%。
面對武漢肺炎疫情延燒，廣華表示，雖然中國新型冠狀病毒對整體經濟帶來嚴峻的考驗，復工後的狀況皆須待2月10日後幾日才較為明確，但公司仍持續密切留意疫情發展情況，配合車廠同步積極恢復生產，並採取即時全方位的因應措施以做充分的準備，包含員工管控、工廠全面消毒、防疫措施準備等。
廣華透露，目前農曆年前庫存可應對車廠復工需求，公司也已與供應商、客戶密切聯繫協調訂單後續出貨時程，以期順產後產能分配，盡速拉升各生產基地的產能稼動率，降低所有不確定因素對公司營運帶來風險。
第一季向為農曆新年出貨天數較少，加上短期中國新車銷售市場因肺炎疫情而造成負面衝擊，但廣華認為，整體訂單僅遞延，並未取消，車廠將以加班或調班方式補回產能；另一方面，公司為降低人事與運營相關成本，各生產基地過去2年已積極致力於製程自動化，因此，產能稼動率有望隨著主要客戶拉貨力道而盡速回升，正面挹注整體營運穩健向上。
此外，廣華將持續發揮全球生產基地的接單優勢，農曆過年後已明顯感受到客戶對於廣華同時在中國、墨西哥等多國生產基地分散化的訂單詢問需求，有助集團持續創造未來營運的良好成長動能。
----------------------------------------------------------------------

[1] 投資等級債適成短期資金避風港　股市反彈先待疫情獲控制

▲ 在肺炎疫情持續影響下，金融市場仍將持續波動。（圖／pixabay）
記者李瑞瑾／綜合報導
金融市場因武漢肺炎疫情影響震盪加劇